# 任务转交`Handoffs`
实现多Agent协作的机制之一，让不同专长的Agent能够像团队成员一样传递工作
作用上理解为**路由**

## motivation
场景：假设你正在构建一个客户支持系统。用户可能会问关于订单状态的问题，也可能问关于如何退款的问题，或者只是问一些常见问题（FAQ）。
问题：如果只有一个Agent，你需要给它非常庞杂的指令，告诉它如何处理这三种截然不同的情况，并且还要让它能调用查询订单、处理退款、检索FAQ文档等多种工具。这会让Agent的指令变得异常复杂，难以维护，而且模型在决策时也容易混淆。
Handoffs的解决方案：我们可以创建三个专家Agent：`OrderAgent`、`RefundAgent`和`FAQAgent`，每个Agent只负责自己的领域，指令清晰，工具专一。然后，我们再创建一个分诊Agent (TriageAgent)，它的唯一职责就是判断用户问题的类型，并将任务转交给相应的专家Agent去处理。

从此处出发`Handoffs`核心机制在于**智能的任务路由和委托** 

## Definition
Handoff作为一种特殊的“工具”：从“分诊Agent”（TriageAgent）开始，每次任务交接都像是一次工具调用。
>举例：例如，交接给`RefundAgent`，在`TriageAgent`看来就是调用了一个名为`transfer_to_Refund_Agent`的工具。(实际使用没这么复杂)

`handoffs`参数：创建`TriageAgent`时，通过`handoffs`参数告诉其下游的Agent有哪些。如` handoffs=[agent_a, agent_b]）`，框架会自动为它们生成默认的转交工具名和描述。

`handoff()`函数：对传入的Agent进行交接定制
- 指定目标Agent`agent=...`
- 覆盖默认的工具名称和描述（`tool_name_override`, `tool_description_override`）
- 在转交发生时，执行一个回调函数（`on_handoff`），比如记录日志或预处理数据
- 定义转交时需要LLM提供的额外输入数据（`input_type`）
- 控制传递给目标Agent的对话历史（`input_filter`）

## Design & Implementation
### 基本用法（直接传入Agent列表）
```python
# ... (LLM配置, history_tutor_agent, math_tutor_agent 定义) ...

# 分诊Agent知道它可以将任务交给历史或数学导师
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, math_tutor_agent], # <--- 直接传入Agent列表
    model=llm,
)

# ... (运行代码: Runner.run(triage_agent, "法国的首都是哪里？")) ...
```
运行流程：
>待补充

### 通过`handoff()`函数定制移交
以下例子为`handoff()`定制交接给`math_tutor_agent`的用法：
```python
# ... (LLM配置, history_tutor_agent, math_tutor_agent 定义) ...

# 定义一个在交接发生时会被调用的函数
def on_handoff(ctx: RunContextWrapper[None]):
    print("Handoff called")

# 使用 handoff() 函数包装 math_tutor_agent
handoff_obj = handoff(
    agent=math_tutor_agent, # 目标Agent
    on_handoff=on_handoff, # 注册回调函数
    tool_name_override="custom_handoff_tool", # 自定义工具名
    tool_description_override="Custom description", # 自定义描述
)

# 分诊Agent现在看到两个转交选项：一个是默认的历史导师，一个是定制的数学导师
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, handoff_obj], # <--- 传入Agent和定制的Handoff对象
    model=llm,
)

# ... (运行代码: Runner.run(triage_agent, "法国的首都是哪里？")) ...
```
运行流程：
>待补充

### 转交时传递额外输入（`input_type`和`on_handoff`回调）
以下例子为`TriageAgent`提供额外的转交信息（`input_type`），并在`on_handoff`回调中使用这些信息：
```python
# ... (LLM配置, history_tutor_agent, math_tutor_agent 定义) ...

# 1. 定义一个Pydantic模型来规范额外输入的数据结构
class EscalationData(BaseModel):
    reason: str

# 2. 定义on_handoff回调，这次它接收一个额外的input_data参数
#    类型提示必须与input_type匹配！
async def on_handoff(ctx: RunContextWrapper[None], input_data: EscalationData):
    print(f"Escalation agent called with reason: {input_data.reason}")

# 3. 使用handoff()配置转交，并指定input_type
handoff_obj = handoff(
    agent=math_tutor_agent,
    on_handoff=on_handoff, # 仍然可以有回调
    tool_name_override="custom_handoff_tool", # 保持自定义名称
    tool_description_override="Custom description",
    input_type=EscalationData, # <--- 告诉框架，调用此工具时需要提供EscalationData类型的数据
)

# 4. 修改分诊Agent的指令，引导它在调用工具时提供reason
triage_agent = Agent(
    name="Triage Agent",
    instructions=(
        "你必须根据用户的问题类型，使用以下工具之一进行转接：\n"
        "- 如果是历史问题，转接给历史导师。\n"
        "- 如果是数学问题，使用 custom_handoff_tool 转接给数学导师，并说明转接原因(reason)。\n" # <--- 引导提供reason
        "不要自己回答问题，必须转接！"
    ),
    handoffs=[history_tutor_agent, handoff_obj],
    model=llm,
)

# ... (运行代码: Runner.run(triage_agent, "如何解决鸡兔同笼问题?")) ...
```
运行流程：
>待补充

### 控制传递的对话历史（`input_filter`）
默认情况下，任务交接，目标Agent会接受完整的对话历史记录，在特定情况进行对话历史控制：
- 隐私/安全：敏感信息等
- 效率/成本：减少历史记录，减少Token消耗
- 专注度：减少Agent过度关注其他内容，focus on the task at hand

使用：
`input_filter`参数允许提供一个函数传入，函数在交接时调用。传入函数：
- 接收当前的对话历史记录（`List[Message]`）
- 返回过滤后的历史记录（`List[Message]`）
- 它接收原始的对话历史 (`HandoffInputData`)，并返回一个新的、经过修改的`HandoffInputData`，用于传递给目标Agent。

```python
from agents.extensions import handoff_filters # 导入预定义的过滤器

# ... (LLM配置, history_tutor_agent, math_tutor_agent 定义) ...

handoff_obj = handoff(
    agent=math_tutor_agent,
    # 使用预定义的过滤器，移除历史记录中所有的工具调用信息
    input_filter=handoff_filters.remove_all_tools, # <--- 传入过滤器函数
)

triage_agent = Agent(
    name="Triage Agent",
    # 增加了一个基于天气的复杂规则，并添加了get_weather工具
    instructions="You determine which agent to use based on the user's homework question, on sunny days, you can let the math_tutor_agent do history homework.",
    handoffs=[history_tutor_agent, handoff_obj],
    tools=[get_weather], # <--- 添加天气工具
    model=llm,
)

# ... (运行代码: Runner.run(triage_agent, "法国的首都是哪里？")) ...
```
运行流程：
>待补充

### 推荐提示词
通过`RRECOMMENDED_PROMPT_PREFIX`预先设置好对多智能体系统和转交机制解释的系统提示文本片段，辅助LLM理解“任务交接”概念，增加准确调用转交请求概率。

将`RRECOMMENDED_PROMPT_PREFIX`添加到`Agent`的`instructions`中，例如：
```python
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX

triage_agent = Agent(
    name="Triage Agent",
    instructions=f"""{RECOMMENDED_PROMPT_PREFIX} 
# ^^^ 把推荐前缀放在这里 ^^^

You determine which agent to use based on the user's homework question. 
# ... (你的其他指令) ...
""",
    handoffs=[history_tutor_agent, math_tutor_agent],
    model=llm,
)
```
这段前缀告诉LLM它在一个多智能体系统中，转交是通过调用特定名称的函数（工具）来实现的，并且在与用户对话时不要提及这些内部的转交过程，以保持交互的流畅性。

## Application
- 客服路由
- 复杂任务拆解：配置项目经理Agent分发：数据收集、分析、报告书写等子任务
- 权限控制：根据用户身份或任务敏感度，将任务转交给不同权限级别的Agent

局限性：
- 交接开销大，每次交接涉及一次LLM调用，增加延迟与成本
- 上下文丢失风险
- 循环交接风险：如果Agent的指令和`handoff_description`设计不当，可能会出现任务在几个Agent之间来回传递、无法解决的死循环。

>待补充：循环交接的解决方法